In [2]:
import xarray as xr
import numpy as np
import pandas as pd
from gen_random_query import gen_random_query
lat_range = np.arange(-90, 90.1, 0.25)
lon_range = np.arange(-180, 180.1, 0.25)
q = gen_random_query()
print(q)

{'min_lat': 39, 'max_lat': 89, 'min_lon': 109, 'max_lon': 110, 'start_datetime': '2006-10-22', 'end_datetime': '2013-02-20', 'time_resolution': 'month', 'time_agg_method': 'mean'}


In [4]:

def time_resolution_to_freq(time_resolution):
    if time_resolution == "hour":
        return "h"
    elif time_resolution == "day":
        return "D"
    elif time_resolution == "month":
        return "ME"
    elif time_resolution == "year":
        return "YE"
    else:
        raise ValueError("Invalid time_resolution")


def gen_xarray_from_query(query):
    lat_start = lat_range.searchsorted(query["min_lat"], side="left")
    lat_end = lat_range.searchsorted(query["max_lat"], side="right")
    lon_start = lon_range.searchsorted(query["min_lon"], side="left")
    lon_end = lon_range.searchsorted(query["max_lon"], side="right")
    ds_query = xr.Dataset()
    ds_query["latitude"] = lat_range[lat_start:lat_end]
    ds_query["longitude"] = lon_range[lon_start:lon_end]
    ds_query["time"] = pd.date_range(
        start=query["start_datetime"], end=query["end_datetime"], freq=time_resolution_to_freq(query["time_resolution"])
    )
    return ds_query


def gen_xarray_from_meta(row):
    lat_start = lat_range.searchsorted(row["min_lat"], side="left")
    lat_end = lat_range.searchsorted(row["max_lat"], side="right")
    lon_start = lon_range.searchsorted(row["min_lon"], side="left")
    lon_end = lon_range.searchsorted(row["max_lon"], side="right")
    ds = xr.Dataset()
    ds["latitude"] = lat_range[lat_start:lat_end]
    ds["longitude"] = lon_range[lon_start:lon_end]
    ds["time"] = pd.date_range(
        start=row["start_datetime"],
        end=row["end_datetime"],
        freq=time_resolution_to_freq(row["resolution"]),
    )
    return ds


def get_relevant_meta(df, query):
    df_relevant = df[
        (df["resolution"] == query["time_resolution"])
        & (df["min_lat"] <= query["max_lat"])
        & (df["max_lat"] >= query["min_lat"])
        & (df["min_lon"] <= query["max_lon"])
        & (df["max_lon"] >= query["min_lon"])
        & (pd.to_datetime(df["start_datetime"]) <= pd.to_datetime(query["end_datetime"]))
        & (pd.to_datetime(df["end_datetime"]) >= pd.to_datetime(query["start_datetime"]))
    ]
    return df_relevant


# def is_overlap(q_min, q_max, m_min, m_max):
#     """return if q overlaps with m"""
#     return q_min <= m_max and m_min <= q_max


def mask_query_with_meta(ds_query, ds_meta):
    return (
        ds_query["latitude"].isin(ds_meta["latitude"])
        & ds_query["longitude"].isin(ds_meta["longitude"])
        & ds_query["time"].isin(ds_meta["time"])
    )



In [7]:
df = pd.read_csv("metadata.csv")
df_relevant = get_relevant_meta(df, q)
df_relevant

# print(f"\nrelevant meta: {df_relevant.shape[0]}\n")

,min_lat,max_lat,min_lon,max_lon,start_datetime,end_datetime,resolution,file_name
20,30,90,60,120,2000-01-01,2009-12-31,month,file_20.nc
23,30,90,60,120,2010-01-01,2019-12-31,month,file_23.nc


In [14]:
ds_query = gen_xarray_from_query(q)
false_mask = xr.DataArray(
    np.zeros((ds_query.sizes["latitude"], ds_query.sizes["longitude"], ds_query.sizes["time"]), dtype=bool),
    dims=["latitude", "longitude", "time"],
)

for index, row in df_relevant.iterrows():
    print(row.to_dict())
    ds_meta = gen_xarray_from_meta(row)
    mask = mask_query_with_meta(ds_query, ds_meta)
    false_mask = false_mask | mask

{'min_lat': 30, 'max_lat': 90, 'min_lon': 60, 'max_lon': 120, 'start_datetime': '2000-01-01', 'end_datetime': '2009-12-31', 'resolution': 'month', 'file_name': 'file_20.nc'}
{'min_lat': 30, 'max_lat': 90, 'min_lon': 60, 'max_lon': 120, 'start_datetime': '2010-01-01', 'end_datetime': '2019-12-31', 'resolution': 'month', 'file_name': 'file_23.nc'}


In [15]:
false_mask

<xarray.DataArray (latitude: 201, longitude: 5, time: 76)> Size: 76kB
array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       ...,

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])
Coordinates:
  * latitude   (latitude) float64 2kB 39.0 39.25 39.5 39.75 ... 88.5 88.75 89.0
  * longitude  (longitude) float64 40B 109.0 109.2 109.5 109.8 110.0
  * time       (time) datetime64[ns] 608B 2006-10-31 2006-11-30 ... 2013-01-31

In [17]:
q

{'min_lat': 39,
 'max_lat': 89,
 'min_lon': 109,
 'max_lon': 110,
 'start_datetime': '2006-10-22',
 'end_datetime': '2013-02-20',
 'time_resolution': 'month',
 'time_agg_method': 'mean'}

In [16]:
ds_query

<xarray.Dataset> Size: 2kB
Dimensions:    (latitude: 201, longitude: 5, time: 76)
Coordinates:
  * latitude   (latitude) float64 2kB 39.0 39.25 39.5 39.75 ... 88.5 88.75 89.0
  * longitude  (longitude) float64 40B 109.0 109.2 109.5 109.8 110.0
  * time       (time) datetime64[ns] 608B 2006-10-31 2006-11-30 ... 2013-01-31
Data variables:
    *empty*

In [19]:
ds_query["latitude"]

<xarray.DataArray 'latitude' (latitude: 201)> Size: 2kB
array([39.  , 39.25, 39.5 , ..., 88.5 , 88.75, 89.  ])
Coordinates:
  * latitude  (latitude) float64 2kB 39.0 39.25 39.5 39.75 ... 88.5 88.75 89.0

In [20]:
ds_meta["latitude"]

<xarray.DataArray 'latitude' (latitude: 241)> Size: 2kB
array([30.  , 30.25, 30.5 , ..., 89.5 , 89.75, 90.  ])
Coordinates:
  * latitude  (latitude) float64 2kB 30.0 30.25 30.5 30.75 ... 89.5 89.75 90.0

In [18]:
ds_query["latitude"].isin(ds_meta["latitude"])

<xarray.DataArray 'latitude' (latitude: 201)> Size: 201B
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])
Coordinates:
  * latitude  (latitude) float64 2kB 39.0 39.25 39.5 39.75 ... 88.5 88.75 89.0

In [22]:
ds_query["longitude"].isin(ds_meta["longitude"])

<xarray.DataArray 'longitude' (longitude: 5)> Size: 5B
array([ True,  True,  True,  True,  True])
Coordinates:
  * longitude  (longitude) float64 40B 109.0 109.2 109.5 109.8 110.0

In [24]:
(ds_query["latitude"].isin(ds_meta["latitude"])
        & ds_query["longitude"].isin(ds_meta["longitude"]))

<xarray.DataArray (latitude: 201, longitude: 5)> Size: 1kB
array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])
Coordinates:
  * latitude   (latitude) float64 2kB 39.0 39.25 39.5 39.75 ... 88.5 88.75 89.0
  * longitude  (longitude) float64 40B 109.0 109.2 109.5 109.8 110.0